In [1]:
import pandas as pd

train_values = pd.read_csv('./data/train_values.csv') 
train_labels = pd.read_csv('./data/train_labels.csv') 

df = train_values.merge(train_labels, on='row_id', how='inner')
df['co_applicant'] = df['co_applicant'].astype(int)

# keep columns 

,loan_type,property_type,loan_purpose,occupancy,loan_amount,preapproval,msa_md,state_code,county_code,applicant_ethnicity,...,property_type_str,loan_purpose_str,occupancy_str,preapproval_str,applicant_ethnicity_str,applicant_race_str,applicant_sex_str,msa_md_str,state_code_str,county_code_str
0,3,1,1,1,-0.647616,3,18,37,246,2,...,1 to 4 Family - non manufactured,Home purchase,Owner Occupied,NaN,Not Hispanic,White,Male,18.0,37,246
1,1,1,3,1,-0.145684,3,369,52,299,1,...,1 to 4 Family - non manufactured,Refinancing,Owner Occupied,NaN,Hispanic,White,Male,369.0,52,299
2,2,1,3,1,-0.215397,3,16,10,306,2,...,1 to 4 Family - non manufactured,Refinancing,Owner Occupied,NaN,Not Hispanic,White,Male,16.0,10,306
3,1,1,1,1,-0.252577,1,305,47,180,2,...,1 to 4 Family - non manufactured,Home purchase,Owner Occupied,Requested,Not Hispanic,White,Male,305.0,47,180
4,1,1,1,1,0.444551,3,24,37,20,2,...,1 to 4 Family - non manufactured,Home purchase,Owner Occupied,NaN,Not Hispanic,African American,Female,24.0,37,20


In [2]:
# scale the numeric columns 

from sklearn.preprocessing import RobustScaler

# float columns to consider
numeric_cols =[ 'loan_amount', 'applicant_income', 'population', 'ffiecmedian_family_income',
               'number_of_owner-occupied_units', 'number_of_1_to_4_family_units']


# use robust scaler for outlier numerics
for col in numeric_cols:
 #   utilities.minMax_normalize_df_col(df, col)
    col_copy = df[[ col ]].copy()
    transformer = RobustScaler().fit( col_copy  )
    df[ col ] = transformer.transform( col_copy )

df [ numeric_cols ].head(5)



,loan_amount,applicant_income,population,ffiecmedian_family_income,number_of_owner-occupied_units,number_of_1_to_4_family_units
0,-0.525714,-0.708333,0.453976,-0.442676,-0.735426,0.876115
1,0.091429,-0.250000,0.296024,-0.810610,0.347982,0.346878
2,0.005714,-0.111111,0.413844,0.012281,-0.682810,-0.703667
3,-0.040000,0.416667,0.624816,0.696217,0.829895,0.536174
4,0.817143,-0.055556,0.648748,-0.284005,0.159043,0.088206


In [3]:
# category encoding 
import category_encoders as ce

#property_type_str  , loan_purpose_str  , preapproval_str  , applicant_race_str  , applicant_sex_str  

#drop_cols = [ 'occupancy_str', 'preapproval_str', 'applicant_ethnicity',
#                     'applicant_race', 'applicant_sex', 'county_code','occupancy']

drop_cols = [ 'loan_type_str', 'occupancy_str', 'applicant_ethnicity_str', 'msa_md_str', 'state_code_str', 'county_code_str' ]

df_clean = df.drop(columns=drop_cols)
df_clean.head(5)

ohe = ce.OneHotEncoder(handle_unknown='ignore', use_cat_names=True)
df = ohe.fit_transform(df_clean)
df.head()

ModuleNotFoundError: No module named 'category_encoders'

In [ ]:
# export
#df_clean = df.drop(columns=drop_cols)
df.to_csv('./export/feature_engineered.csv', index=False)

df.head(5)